In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path+"\\Source")

from Source.Agent import Agent, DeltaHedger
from Source.Market import Market, StockGeometricBrownianMotion, EuropeanCallOption

In this notebook, we will research how hedging frequency influences the variance of PnL.

We create a Martingale GBM. We verify the process is a martingale by Monte Carlo Simulation. The initial stock price is 100, and we expect the average of stocks price  is  100 at T=100. We simulate 10000 times and calculate the average.

In [2]:
final_stock_price_list = []
for _ in range(10000):
    stock = StockGeometricBrownianMotion('stock_gbm', 100, -0.02/252, 0.2 / np.sqrt(252))  # Martingale GBM
    for time in range(1, 100):
        stock.evolve(time)
    final_stock_price_list.append(stock.current_value)
print(f'Mean of stock final values: {np.mean(final_stock_price_list):.3f}')
print(f'Std of stock final values: {np.std(final_stock_price_list):.3f}')

Mean of stock final values: 100.053
Std of stock final values: 12.476


Suppose the stock price follows a Martingale GBM process with annual volatility 20%. An option trader holds 10 ATM options with one week expiry. The trader decides to delta hedge her option positions everyday. We will use this example to illustrate her delta hedging process.

In [3]:
stock = StockGeometricBrownianMotion('stock_gbm', 100, -0.02 / 252, 0.2 / np.sqrt(252))
option = EuropeanCallOption('option', [stock], 100, 5)
market = Market([stock, option])

delta_hedging_initial_asset = {'Cash': 10000, 'stock_gbm': 0, 'option': 10}  # agent holds an option
delta_hedging_trader = DeltaHedger('Agent', delta_hedging_initial_asset)

# set up initial hedging
print('\nTime=', 0)
market.mark_current_value_to_record(0)
delta_hedging_trader.generate_delta_hedging_plans(market)
delta_hedging_trader.trade(market, 0, print_log=True)
delta_hedging_trader.evaluate_holding_asset_values(market, print_log=True)
delta_hedging_trader.generate_performance_report(market, 0)

for time in range(1, 7):
    print('\nTime=', time)
    market.evolve(time)
    market.mark_current_value_to_record(time)
    delta_hedging_trader.evaluate_holding_asset_values(market, print_log=True)
    delta_hedging_trader.generate_performance_report(market, time)
    delta_hedging_trader.generate_delta_hedging_plans(market)
    delta_hedging_trader.trade(market, time, print_log=True)

print('\nEnd of Hedging')
delta_hedging_trader.evaluate_holding_asset_values(market, print_log=True)
print(f'\nCumulative PnL: ${delta_hedging_trader.historical_performance[5].cumulative_pnl:.3f}')


Time= 0
Agent: sell 5.0 stock_gbm, Cash + $500.000
Agent, Holding:
Cash: $10500.000
stock_gbm: -5.0 * $100.000 = $-500.000
option: 10 * $1.124 = $11.239
Total: $10011.239

Time= 1
Agent, Holding:
Cash: $10500.000
stock_gbm: -5.0 * $100.720 = $-503.599
option: 10 * $1.409 = $14.094
Total: $10010.494
Agent: sell 1.0 stock_gbm, Cash + $100.720

Time= 2
Agent, Holding:
Cash: $10600.720
stock_gbm: -6.0 * $101.619 = $-609.712
option: 10 * $1.914 = $19.143
Total: $10010.151
Agent: sell 2.0 stock_gbm, Cash + $203.237

Time= 3
Agent, Holding:
Cash: $10803.957
stock_gbm: -8.0 * $103.296 = $-826.365
option: 10 * $3.320 = $33.202
Total: $10010.794
Agent: sell 2.0 stock_gbm, Cash + $206.591

Time= 4
Agent, Holding:
Cash: $11010.548
stock_gbm: -10.0 * $101.417 = $-1014.172
option: 10 * $1.501 = $15.014
Total: $10011.390
Agent: buy 1.0 stock_gbm, Cash - $101.417

Time= 5
Agent, Holding:
Cash: $10909.131
stock_gbm: -9.0 * $103.337 = $-930.034
option: 10 * $3.337 = $33.371
Total: $10012.468
Agent: buy

We compare the pnl differences between 3 delta hedging strategy to hedge a 10 days ATM option:
1. Daily delta hedging
2. Only initial hedging
3. No delta hedging

In [5]:
def simulate_delta_hedge_pnl(num_trails, simulation_time, market_input, delta_hedging_initial_asset, hedging_method):
    final_pnl_list = []
    for _ in range(num_trails):
        delta_hedging_trader = DeltaHedger('Agent', copy.deepcopy(delta_hedging_initial_asset))

        # set up initial hedging
        market = copy.deepcopy(market_input)
        market.mark_current_value_to_record(0)
        if hedging_method in ['daily_delta_hedging', 'initial_delta_hedging']:
            delta_hedging_trader.generate_delta_hedging_plans(market)
            delta_hedging_trader.trade(market, 0)
        delta_hedging_trader.evaluate_holding_asset_values(market)
        delta_hedging_trader.generate_performance_report(market, 0)

        for time in range(1, simulation_time + 1):
            market.evolve(time)
            market.mark_current_value_to_record(time)
            delta_hedging_trader.evaluate_holding_asset_values(market)
            delta_hedging_trader.generate_performance_report(market, time)
            if hedging_method in ['daily_delta_hedging']:
                delta_hedging_trader.generate_delta_hedging_plans(market)
                delta_hedging_trader.trade(market, time)

        final_pnl_list.append(delta_hedging_trader.historical_performance[simulation_time].cumulative_pnl)
    return final_pnl_list


stock = StockGeometricBrownianMotion('stock_gbm', 100, -0.02 / 252, 0.2 / np.sqrt(252))
option = EuropeanCallOption('option', [stock], 100, 10)
market = Market([stock, option])

delta_hedging_initial_asset = {'Cash': 10000, 'stock_gbm': 0, 'option': 10}

delta_hedge_pnl = simulate_delta_hedge_pnl(3000, 10, market,
                                           delta_hedging_initial_asset, 'daily_delta_hedging')
print('ATM Option')
print('Daily Delta Hedger:')
print(f'Average Cumulative PnL: ${np.mean(delta_hedge_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(delta_hedge_pnl):.3f}')
print(f'Max Abs Cumulative PnL: ${np.max(np.abs(delta_hedge_pnl)):.3f}')

initial_delta_hedging_pnl = simulate_delta_hedge_pnl(3000, 10, market,
                                                     delta_hedging_initial_asset, 'initial_delta_hedging')
print('\nTrader who only does Initial Hedge:')
print(f'Average Cumulative PnL: ${np.mean(initial_delta_hedging_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(initial_delta_hedging_pnl):.3f}')
print(f'Max Abs Cumulative PnL: ${np.max(np.abs(initial_delta_hedging_pnl)):.3f}')

non_hedge_pnl = simulate_delta_hedge_pnl(3000, 10, market,
                                         delta_hedging_initial_asset, 'no_delta_hedging')
print('\nTrader who Never Hedge:')
print(f'Average Cumulative PnL: ${np.mean(non_hedge_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(non_hedge_pnl):.3f}')
print(f'Max Abs Cumulative PnL: ${np.max(np.abs(non_hedge_pnl)):.3f}')

ATM Option
Daily Delta Hedger:
Average Cumulative PnL: $0.001
Std Cumulative PnL: $4.212
Max Abs Cumulative PnL: $20.647



Trader who only does Initial Hedge:
Average Cumulative PnL: $0.242
Std Cumulative PnL: $12.189
Max Abs Cumulative PnL: $58.662



Trader who Never Hedge:
Average Cumulative PnL: $-0.110
Std Cumulative PnL: $23.496
Max Abs Cumulative PnL: $122.120


According to simulation results, initial delta hedging can reduce the half of delta risk from an ATM option. Daily delta hedging can reduce 85% of delta risk. Delta hedging can also reduce tail risk significantly

Then we do a similar simulation for OTM options and ATM options. We can compare how diffusion risk (std) and tail risk can reduce through delta hedging.

In [7]:
stock = StockGeometricBrownianMotion('stock_gbm', 100, -0.02 / 252, 0.2 / np.sqrt(252))
option = EuropeanCallOption('option', [stock], 105, 10)
market = Market([stock, option])

delta_hedging_initial_asset = {'Cash': 10000, 'stock_gbm': 0, 'option': 10}

delta_hedge_pnl = simulate_delta_hedge_pnl(3000, 10, market,
                                           delta_hedging_initial_asset, 'daily_delta_hedging')
print('OTM Option')
print('Daily Delta Hedger:')
print(f'Average Cumulative PnL: ${np.mean(delta_hedge_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(delta_hedge_pnl):.3f}')
print(f'Max Abs Cumulative PnL: ${np.max(np.abs(delta_hedge_pnl)):.3f}')

initial_delta_hedging_pnl = simulate_delta_hedge_pnl(3000, 10, market,
                                                     delta_hedging_initial_asset, 'initial_delta_hedging')
print('\nTrader who only does Initial Hedge:')
print(f'Average Cumulative PnL: ${np.mean(initial_delta_hedging_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(initial_delta_hedging_pnl):.3f}')
print(f'Max Abs Cumulative PnL: ${np.max(np.abs(initial_delta_hedging_pnl)):.3f}')

non_hedge_pnl = simulate_delta_hedge_pnl(3000, 10, market,
                                         delta_hedging_initial_asset, 'no_delta_hedging')
print('\nTrader who Never Hedge:')
print(f'Average Cumulative PnL: ${np.mean(non_hedge_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(non_hedge_pnl):.3f}')
print(f'Max Abs Cumulative PnL: ${np.max(np.abs(non_hedge_pnl)):.3f}')

stock = StockGeometricBrownianMotion('stock_gbm', 100, -0.02 / 252, 0.2 / np.sqrt(252))
option = EuropeanCallOption('option', [stock], 95, 10)
market = Market([stock, option])

delta_hedging_initial_asset = {'Cash': 10000, 'stock_gbm': 0, 'option': 10}

delta_hedge_pnl = simulate_delta_hedge_pnl(3000, 10, market,
                                           delta_hedging_initial_asset, 'daily_delta_hedging')
print('\n\nITM Option')
print('Daily Delta Hedger:')
print(f'Average Cumulative PnL: ${np.mean(delta_hedge_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(delta_hedge_pnl):.3f}')
print(f'Max Abs Cumulative PnL: ${np.max(np.abs(delta_hedge_pnl)):.3f}')

initial_delta_hedging_pnl = simulate_delta_hedge_pnl(3000, 10, market,
                                                     delta_hedging_initial_asset, 'initial_delta_hedging')
print('\nTrader who only does Initial Hedge:')
print(f'Average Cumulative PnL: ${np.mean(initial_delta_hedging_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(initial_delta_hedging_pnl):.3f}')
print(f'Max Abs Cumulative PnL: ${np.max(np.abs(initial_delta_hedging_pnl)):.3f}')

non_hedge_pnl = simulate_delta_hedge_pnl(3000, 10, market,
                                         delta_hedging_initial_asset, 'no_delta_hedging')
print('\nTrader who Never Hedge:')
print(f'Average Cumulative PnL: ${np.mean(non_hedge_pnl):.3f}')
print(f'Std Cumulative PnL: ${np.std(non_hedge_pnl):.3f}')
print(f'Max Abs Cumulative PnL: ${np.max(np.abs(non_hedge_pnl)):.3f}')

OTM Option
Daily Delta Hedger:
Average Cumulative PnL: $-0.033
Std Cumulative PnL: $2.837
Max Abs Cumulative PnL: $17.467



Trader who only does Initial Hedge:
Average Cumulative PnL: $0.016
Std Cumulative PnL: $7.305
Max Abs Cumulative PnL: $129.081



Trader who Never Hedge:
Average Cumulative PnL: $-0.070
Std Cumulative PnL: $8.422
Max Abs Cumulative PnL: $149.403




ITM Option
Daily Delta Hedger:
Average Cumulative PnL: $0.010
Std Cumulative PnL: $2.491
Max Abs Cumulative PnL: $19.785



Trader who only does Initial Hedge:
Average Cumulative PnL: $0.064
Std Cumulative PnL: $6.113
Max Abs Cumulative PnL: $60.406



Trader who Never Hedge:
Average Cumulative PnL: $-1.782
Std Cumulative PnL: $35.332
Max Abs Cumulative PnL: $128.488


For OTM options, initial delta hedge helps little. This is intuitive become delta for OTM options is small. Initial delta hedge helps more for the ITM options.